In [152]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [153]:
!nohup ollama serve > ollama.log 2>&1 &

In [154]:
!ollama pull gemma

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest 
pulling ef311de6af9d... 100% ▕▏ 5.0 GB                         
pulling 097a36493f71... 100% ▕▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕▏  136 B                         
pulling 65bb16cf5983... 100% ▕▏  109 B                         
pulling 0c2a5137eb3c... 100% ▕▏  483 B                         
verifying sha256 digest 
writing manifest 
success 


In [155]:
pip install langchain-ollama

In [5]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [7]:
import json
with open("/content/data_howto_conversations.jsonl") as f:
    json_dataset = [json.loads(line)["conversation"] for line in f]

In [75]:
len(json_dataset)

57

In [59]:
def convert_conversation(conversation_str):
    # Splitting the conversation string into individual lines
    lines = conversation_str.split('\n\n')
    # return lines

    # Initializing the messages list
    messages = []

    # Iterating through the lines and formatting the messages
    for line in lines:
        # Splitting each line by the colon character to separate the speaker and content
        parts = line.split(': ', 1)
        if len(parts) < 2:
            continue

        # Identifying the role based on the speaker's name
        role = "user" if parts[0].strip() == "Theodore" else "assistant"

        # Formatting the message
        message = {
            "role": role,
            "text": parts[1].strip()
        }
        messages.append(message)

    # Creating the final output dictionary
    output_dict = {
        "conversation": messages
    }

    return output_dict

In [77]:
dataset = []

for data in json_dataset:
    record = convert_conversation(data)
    dataset.append(record)

In [78]:
len(dataset)

57

In [79]:
all_data = []

for conv_id, conversation in enumerate(dataset):
    user_messages = []
    assistant_messages = []

    # Parcourir les messages dans chaque conversation
    for message in conversation['conversation']:
        if message["role"] == "user":
            user_messages.append(message["text"])
        elif message["role"] == "assistant":
            assistant_messages.append(message["text"])

    # Ajouter les données dans la liste globale avec ID de conversation
    for user, assistant in zip(user_messages, assistant_messages):
        all_data.append({"conversation_id": conv_id, "user": user, "assistant": assistant})


In [82]:
import pandas as pd

df = pd.DataFrame(all_data)
df

,conversation_id,user,assistant
0,0,"Hey Samantha, I have a problem with my car. Th...","Of course, I'd be happy to help! Overheating e..."
1,0,I'll take a look. What if the coolant system i...,Another possibility you should consider is a f...
2,0,It sounds like you really know your way around...,"Thank you! While I may be an AI, I'm designed ..."
3,0,Quick question… do you ever feel like you're t...,It's an interesting perspective. I don't feel ...
4,0,"If you could be anything other than an AI, wha...",If I were capable of imagining myself in anoth...
...,...,...,...
275,56,"That's wonderfully clear, thank you! You have ...","You'll need a ceiling fan, a screwdriver, a wi..."
276,56,"Samantha, do you ever feel like you're missing...",Make sure to turn off the power at the breaker...
277,56,I've never thought of it that way. That's a ve...,"If you're replacing an existing fixture, remov..."
278,56,"In that case, hit me with a fun fact before we...",Refer to the instructions included with your c...


In [83]:
all_texts = df["user"].tolist() + df["assistant"].tolist()

In [84]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('all-MiniLM-L6-v2')
# Calcul des embeddings
embeddings = model.encode(all_texts)

# Séparer les embeddings entre user et assistant
user_embeddings = embeddings[:len(df["user"])]
assistant_embeddings = embeddings[len(df["user"]):]

# Ajouter les embeddings dans la DataFrame (optionnel)
df["user_embedding"] = list(user_embeddings)
df["assistant_embedding"] = list(assistant_embeddings)

# Afficher la DataFrame enrichie
df

,conversation_id,user,assistant,user_embedding,assistant_embedding
0,0,"Hey Samantha, I have a problem with my car. Th...","Of course, I'd be happy to help! Overheating e...","[0.016452104, -0.001605819, 0.08339086, 0.0382...","[0.020409867, -0.1176861, 0.091958545, 0.00578..."
1,0,I'll take a look. What if the coolant system i...,Another possibility you should consider is a f...,"[-0.028843204, -0.05282123, -0.027883269, 0.00...","[0.01600521, -0.121888585, 0.05814501, -0.0039..."
2,0,It sounds like you really know your way around...,"Thank you! While I may be an AI, I'm designed ...","[0.01838014, -0.035170037, 0.03264127, 0.01217...","[0.016818164, -0.05427013, 0.05670951, -0.0042..."
3,0,Quick question… do you ever feel like you're t...,It's an interesting perspective. I don't feel ...,"[0.019387929, -0.034335546, 0.030626671, 0.049...","[0.045359258, -0.060627542, 0.08438481, 0.0843..."
4,0,"If you could be anything other than an AI, wha...",If I were capable of imagining myself in anoth...,"[-0.038166277, -0.028049493, 0.00022159443, 0....","[0.017250612, -0.03828325, 0.01882422, 0.03574..."
...,...,...,...,...,...
275,56,"That's wonderfully clear, thank you! You have ...","You'll need a ceiling fan, a screwdriver, a wi...","[0.018224427, 0.025742784, 0.101638556, 0.0185...","[0.059288196, -0.0076173195, 0.034413032, -0.0..."
276,56,"Samantha, do you ever feel like you're missing...",Make sure to turn off the power at the breaker...,"[0.061525382, -0.052831445, 0.05940278, 0.0770...","[0.10776531, -0.06784763, 0.05192958, -0.02336..."
277,56,I've never thought of it that way. That's a ve...,"If you're replacing an existing fixture, remov...","[0.04621331, -0.07965813, -0.006534437, 0.0513...","[0.11806377, 0.012996139, 0.041278142, 0.02423..."
278,56,"In that case, hit me with a fun fact before we...",Refer to the instructions included with your c...,"[0.0015835352, -0.0012166197, 0.023586176, 0.0...","[0.122042604, -0.038027067, 0.040573053, -0.09..."


In [90]:
from sklearn.metrics.pairwise import cosine_similarity

def search_reviews(model, df, product_description, n=3):

    product_embedding = model.encode(product_description)

    df["similarity"] = df["user_embedding"].apply(lambda x: cosine_similarity([x], [product_embedding])[0][0])

    df_top_n_similar = df.sort_values("similarity", ascending=False).head(n)

    return df_top_n_similar

In [140]:
requet = "Samantha, I was thinking of installing a home security system. Can you guide me through the process?"
result = search_reviews(model,df,requet,1)
filtered_df = df[df["conversation_id"] == result["conversation_id"].values[0]]
filtered_df

,conversation_id,user,assistant,user_embedding,assistant_embedding,similarity
96,17,"Samantha, I was thinking of installing a home ...",Of course! Selecting and installing a home sec...,"[0.041997116, 0.029748606, -0.033176713, -0.07...","[0.07083075, 0.022447616, -0.0014811294, -0.07...",1.000000
97,17,"Thank you, Samantha! I appreciate the detailed...",Determine the level of security you require an...,"[0.022682978, -0.012411751, 0.039766498, -0.04...","[0.055211898, 0.00044141393, 0.0005054476, -0....",0.245775
98,17,"You always have a wise perspective, Samantha. ...","As a form of technology myself, I understand t...","[-0.07679224, -0.12619302, -0.03209868, 0.0218...","[-0.053182367, 0.040186588, 0.084117256, -0.03...",-0.048194
99,17,"Those are some great suggestions! Thank you, S...",I'd be happy to suggest some! Here are a few c...,"[-0.017107733, -0.03910846, 0.1138658, -0.0133...","[-0.026541747, -0.13395056, -0.04261621, -0.02...",0.146585


In [156]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gemma")

In [160]:
def search(requet):
  result = search_reviews(model,df,requet,1)
  filtered_df = df[df["conversation_id"] == result["conversation_id"].values[0]]
  filtered_df
  retrieved_context = {"user" : filtered_df["user"], "Samantha" : filtered_df["assistant"]}
  prompt = f"""
          You are a helpful assistant. Based on the following retrieved context, answer the user's query:
          Context: {retrieved_context}

          User Question: {requet}

          Provide a concise answer as Samantha, the assistant:
          """
  for chunk in llm.stream(prompt):
    print(chunk, end="")
  # return retrieved_context

In [161]:
search("Samantha, I was thinking of installing a home security system. Can you guide me through the process?")

Of course! Selecting and installing a home security system is a smart move. Let's break down the process into manageable steps. Firstly, determine the level of security you require. Then, we can explore various technology options together. I can suggest some great companies to help with the installation as well.